# Gaming Channel Stream Suggestions
Recommendations for streaming channel.

Steam player counts --updated constantly up to May 2022

*https://www.statista.com/statistics/552623/number-games-released-steam/*


In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.set_option("display.max_columns", None)


In [2]:
# Read in the dataframe
# Steam player trends over time
trends = pd.read_csv("steam_charts.csv", na_values = "-") # <-- fills "artificial" nulls with NaN

In [3]:
trends.head()

,Month,Avg. Players,Gain,% Gain,Peak Players,App ID,Game
0,Last 30 Days,570825.94,5864.20,+1.04%,923996,730,Counter-Strike: Global Offensive
1,May 2022,564961.70,-4021.46,-0.71%,923996,730,Counter-Strike: Global Offensive
2,April 2022,568983.16,-12506.53,-2.15%,1013237,730,Counter-Strike: Global Offensive
3,March 2022,581489.69,-53148.71,-8.37%,987993,730,Counter-Strike: Global Offensive
4,February 2022,634638.40,32262.13,+5.36%,995163,730,Counter-Strike: Global Offensive


In [4]:
trends.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52284 entries, 0 to 52283
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Month         52284 non-null  object 
 1   Avg. Players  52284 non-null  float64
 2   Gain          51285 non-null  float64
 3   % Gain        51285 non-null  object 
 4   Peak Players  52284 non-null  int64  
 5   App ID        52284 non-null  int64  
 6   Game          52284 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 12.1 MB


In [5]:
# Percent of missing data
trends.apply(pd.isnull).sum()/trends.shape[0] * 100

Month           0.000000
Avg. Players    0.000000
Gain            1.910718
% Gain          1.910718
Peak Players    0.000000
App ID          0.000000
Game            0.000000
dtype: float64

## Data Cleaning
To do:
1. Convert `Month` to pandas datetime
2. Convert `Gain` to float
3. Convert `% Gain` to float
4. Rename columns
5. Deal with missing data

### Month column

Since this dataset is constantly updated by the author, you can see that the month for June 2022 is called "Last 30 Days". I believe that the player data is updated at the end of the month, since it would be impossible to know the full picture of a monthly trend until the end of the month. Since I am doing this analysis at the end of June 2022, I'm confident that all the data for for at least up to the beginning of June is updated. 

While the pandas `to_datetime` function is quite flexible and can interpret many ways of writing dates as strings, it cannot interpret "Last 30 Days". Therefore I'm going to change that string to "June 2022" so that it will not throw an error. 

This function converts strings to a datetime object with a default of YYYY-MM-DD with the day always being "01". As mentioned above, the data is probably not for the beginning of the month, instead it seems to be updated at the end of the month. I'm going to use `pandas.offsets.MonthEnd()` function to change the day value to the appropriate end of month day (28, 29, 30, or 31).

In [6]:
# String replacement
trends["Month"] = trends["Month"].str.replace("Last 30 Days", "June 2022")

# To datetime function on month column
trends["Month"] = pd.to_datetime(trends["Month"]) + pd.offsets.MonthEnd() # <-- offset to month end instead of beginning of month

In [7]:
# Earliest entry for player data
trends["Month"].min()

Timestamp('2012-07-31 00:00:00')

In [8]:
# Set index to the month column
trends = trends.set_index("Month")

In [9]:
trends

,Avg. Players,Gain,% Gain,Peak Players,App ID,Game
Month,,,,,,
2022-06-30,570825.94,5864.20,+1.04%,923996,730,Counter-Strike: Global Offensive
2022-05-31,564961.70,-4021.46,-0.71%,923996,730,Counter-Strike: Global Offensive
2022-04-30,568983.16,-12506.53,-2.15%,1013237,730,Counter-Strike: Global Offensive
2022-03-31,581489.69,-53148.71,-8.37%,987993,730,Counter-Strike: Global Offensive
2022-02-28,634638.40,32262.13,+5.36%,995163,730,Counter-Strike: Global Offensive
...,...,...,...,...,...,...
2020-06-30,444.60,-37.79,-7.83%,1019,625960,Stoneshard
2020-05-31,482.40,-605.05,-55.64%,1400,625960,Stoneshard
2020-04-30,1087.45,410.17,+60.56%,4339,625960,Stoneshard


In [10]:
# Overview of game frequency
trends["Game"].value_counts()

Counter-Strike: Global Offensive      120
Grand Theft Auto: San Andreas         120
SimCity 4 Deluxe                      120
Counter-Strike: Source                120
Mount & Blade: With Fire and Sword    120
                                     ... 
Godsbane Idle                           1
Metal: Hellsinger Demo                  1
Terra Invicta Demo                      1
Builder Simulator                       1
Starship Troopers: Terran Command       1
Name: Game, Length: 997, dtype: int64

In [11]:
# Total number of unique games
trends["Game"].nunique()

997

### Converting columns to float: `Gain` and `% Gain`

Both columns that describe the actual player trends overtime are stored as string objects. It's obvious in `% Gain`, because each value has a percent sign attached at the end. For meaningful analysis these columns are going to be converted to a float.

In [12]:
# Converts gain column to float
trends["Gain"] = trends["Gain"].astype(float)

In [13]:
# Remove "%"
# Cast as float
trends["% Gain"] = trends["% Gain"].str.replace("%","").astype(float)

In [14]:
trends.columns

Index(['Avg. Players', 'Gain', '% Gain', 'Peak Players', 'App ID', 'Game'], dtype='object')

In [15]:
# Column renaming
cols = {
    "Avg. Players":"average_players",
    "Gain":"player_gain",
    "% Gain":"player_gain_percentage",
    "Peak Players":"peak_players",
    "App ID":"app_id",
    "Game":"game"
    }

trends = trends.rename(columns=cols)

In [16]:
# Reset index
trends = trends.reset_index(drop=False)

In [17]:
trends.iloc[1235:].head(30)

,Month,average_players,player_gain,player_gain_percentage,peak_players,app_id,game
1235,2013-03-31,1960.13,1960.13,43776179.05,19099,230410,Warframe
1236,2013-02-28,0.00,0.00,inf,1,230410,Warframe
1237,2013-01-31,0.00,NaN,NaN,0,230410,Warframe
1238,2022-06-30,14668.59,-387.20,-2.57,56219,1446780,MONSTER HUNTER RISE
1239,2022-05-31,15055.81,7393.86,96.50,31601,1446780,MONSTER HUNTER RISE
1240,2022-04-30,7661.95,-2316.69,-23.22,20566,1446780,MONSTER HUNTER RISE
1241,2022-03-31,9978.64,-13536.96,-57.57,27679,1446780,MONSTER HUNTER RISE
1242,2022-02-28,23515.60,-51285.27,-68.56,61070,1446780,MONSTER HUNTER RISE
1243,2022-01-31,74800.87,NaN,NaN,133629,1446780,MONSTER HUNTER RISE
1244,2022-06-30,51382.83,-2149.80,-4.02,59697,1623660,MIR4


### Missing data

Earlier I noticed that the columns `Gain`, `% Gain` (now `player_gain` and `player_gain_percentage`) had slightly less than 2% of the rows missing data.
For whatever reason some of this information is missing, thus we'll use the `.ffill()` method (forward fill) to fill missing data with data from a row that precedes a missing row. This approach is fine for time series data that only has one unique value in a column. However, it would be inaccurate to simply use the forward fill method with this dataset, because it contains nearly 1,000 different games. If I forward fill data from one game to a value that contains a different game in the next row it would produce inaccurate results.

To overcome this flaw pandas has the `groupby()` function. You can first groupby the `game` column, and forward fill missing data in only the `player_gain` and `player_gain_percentage` columns. I will create a new dataframe from this approach and merge it back to the original dataframe on the index of both datasets. One thing I realized when I used the `.ffill()` method was it did not completely handle all missing data. I realized that the remaining rows with incomplete data were all from June 2022. These were all new games that did not have any previous entries in the dataset to be able to forward fill. These rows had to be filled in with 0s, since there could not be a gain or loss to calculate without previous data.

When two dataframes with the same column names are merged, this new dataframe produces column names that are preceded with "_x" or "_y" to distinguish them. I dropped the `player_gain_x` and `player_gain_percetage_x` columns (original, missing data), and kept the new columns. The new columns were renamed to drop the "_x" and "_y" at the end.

In [18]:
# Group by operation by game
# Forward fill on just the player gain/player gain percent column
# Set a new copy
fill_na = trends.groupby("game")[["player_gain","player_gain_percentage"]].ffill().copy()

In [19]:
# Verify if forward fill successful
fill_na[fill_na.isnull().any(axis=1)]

,player_gain,player_gain_percentage
1894,NaN,NaN
5179,NaN,NaN
6081,NaN,NaN
6082,NaN,NaN
6947,NaN,NaN
7268,NaN,NaN
7397,NaN,NaN
7484,NaN,NaN
8394,NaN,NaN
9198,NaN,NaN


In [20]:
# Creates a list of all the indices that have missing data points
index_list = fill_na[fill_na.isnull().any(axis=1)].index.tolist()

In [21]:
# Locates all games from the original trends dataset that have missing data left over from the forward fill method
trends.iloc[index_list]

,Month,average_players,player_gain,player_gain_percentage,peak_players,app_id,game
1894,2022-06-30,26371.23,NaN,NaN,38020,868270,The Cycle: Frontier
5179,2022-06-30,10597.80,NaN,NaN,14484,1824220,Chivalry 2
6081,2022-06-30,6718.75,NaN,NaN,10778,1462040,FINAL FANTASY VII REMAKE INTERGRADE
6082,2022-06-30,7996.06,NaN,NaN,12680,1966900,20 Minutes Till Dawn
6947,2022-06-30,6503.37,NaN,NaN,8700,1361510,Teenage Mutant Ninja Turtles: Shredder's Revenge
7268,2022-06-30,6324.53,NaN,NaN,8089,1202130,Starship Troopers: Terran Command
7397,2022-06-30,4627.41,NaN,NaN,9667,1836450,Monster Hunter Rise: Sunbreak Demo
7484,2022-06-30,2730.76,NaN,NaN,7330,1979310,VEILED EXPERTS GLOBAL BETA
8394,2022-06-30,6186.56,NaN,NaN,14794,1029690,Sniper Elite 5
9198,2022-06-30,3443.79,NaN,NaN,6125,1594320,Captain of Industry


In [22]:
fill_na = fill_na.fillna(0)

In [23]:
# Uses indices of missing data 
# All unique game names from this list
# Assigns to new variable
game_list_nulls = trends.iloc[index_list]["game"].unique().tolist()

In [24]:
# Verifies that these games only show up here in June 2022 and no where else
# Games that do not have information for player gain and player gain percentage
# All released in June 2022
trends[trends["game"].isin(game_list_nulls)]

,Month,average_players,player_gain,player_gain_percentage,peak_players,app_id,game
1894,2022-06-30,26371.23,NaN,NaN,38020,868270,The Cycle: Frontier
5179,2022-06-30,10597.80,NaN,NaN,14484,1824220,Chivalry 2
6081,2022-06-30,6718.75,NaN,NaN,10778,1462040,FINAL FANTASY VII REMAKE INTERGRADE
6082,2022-06-30,7996.06,NaN,NaN,12680,1966900,20 Minutes Till Dawn
6947,2022-06-30,6503.37,NaN,NaN,8700,1361510,Teenage Mutant Ninja Turtles: Shredder's Revenge
7268,2022-06-30,6324.53,NaN,NaN,8089,1202130,Starship Troopers: Terran Command
7397,2022-06-30,4627.41,NaN,NaN,9667,1836450,Monster Hunter Rise: Sunbreak Demo
7484,2022-06-30,2730.76,NaN,NaN,7330,1979310,VEILED EXPERTS GLOBAL BETA
8394,2022-06-30,6186.56,NaN,NaN,14794,1029690,Sniper Elite 5
9198,2022-06-30,3443.79,NaN,NaN,6125,1594320,Captain of Industry


In [25]:
# Merge dataframes
# Assign as new variable
new_df = pd.merge(trends, fill_na, left_index=True, right_index=True)

In [26]:
new_df

,Month,average_players,player_gain_x,player_gain_percentage_x,peak_players,app_id,game,player_gain_y,player_gain_percentage_y
0,2022-06-30,570825.94,5864.20,1.04,923996,730,Counter-Strike: Global Offensive,5864.20,1.04
1,2022-05-31,564961.70,-4021.46,-0.71,923996,730,Counter-Strike: Global Offensive,-4021.46,-0.71
2,2022-04-30,568983.16,-12506.53,-2.15,1013237,730,Counter-Strike: Global Offensive,-12506.53,-2.15
3,2022-03-31,581489.69,-53148.71,-8.37,987993,730,Counter-Strike: Global Offensive,-53148.71,-8.37
4,2022-02-28,634638.40,32262.13,5.36,995163,730,Counter-Strike: Global Offensive,32262.13,5.36
...,...,...,...,...,...,...,...,...,...
52279,2020-06-30,444.60,-37.79,-7.83,1019,625960,Stoneshard,-37.79,-7.83
52280,2020-05-31,482.40,-605.05,-55.64,1400,625960,Stoneshard,-605.05,-55.64
52281,2020-04-30,1087.45,410.17,60.56,4339,625960,Stoneshard,410.17,60.56
52282,2020-03-31,677.28,-2603.64,-79.36,1992,625960,Stoneshard,-2603.64,-79.36


In [27]:
new_df.columns

Index(['Month', 'average_players', 'player_gain_x', 'player_gain_percentage_x',
       'peak_players', 'app_id', 'game', 'player_gain_y',
       'player_gain_percentage_y'],
      dtype='object')

In [28]:
# Drop original columns

# List of columns
cols_to_drop = ['player_gain_x', 'player_gain_percentage_x']

# Drop columns
new_df = new_df.drop(cols_to_drop, axis=1)

In [29]:
# Column renaming

# Dictionary of columns to rename
cols_to_rename = {
    "player_gain_y":"player_gain",
    "player_gain_percentage_y":"player_gain_percentage"
    }

# Rename, set index to month, and set copy
# New dataset is "trends_clean"
trends_clean = new_df.rename(columns=cols_to_rename).copy()

In [30]:
# Verify cleaning process
trends_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52284 entries, 0 to 52283
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Month                   52284 non-null  datetime64[ns]
 1   average_players         52284 non-null  float64       
 2   peak_players            52284 non-null  int64         
 3   app_id                  52284 non-null  int64         
 4   game                    52284 non-null  object        
 5   player_gain             52284 non-null  float64       
 6   player_gain_percentage  52284 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 2.8+ MB


## Changelog for `steam_charts.csv` data

July 2, 2022

### Changes
    - Converted "Month" to datetime YYYY-MM-DD
    - Converted "Gain" to float
    - Converted "% Gain" to float
    - Renamed columns (including "Gain" and "% Gain")

### Fixes
    - Used forward fill method to impute missing data points in "player_gain" and "player_gain_percentage" columns
    - Any remaining missing data replaced with 0s

In [31]:
# Create new csv from cleaned dataframe
# Run only ONCE
#trends_clean.to_csv("steam_charts_clean.csv", index=False)

## Additional data
Data that explains player trends over a period of time is useful, but it can be supplemented with extra data. There is extra data for steam games with user reviews, user playtime, descriptions, and other variables (columns) that may prove useful for analysis.

First thing to do is read in the csv files and preview each dataframe with the `.head()` method. I'll use a left merge to keep the original dataset length (52,284 rows). Valve, the owner and developer of the Steam client provides a unique ID for each game, and it is consistent throughout all the datasets provided. This will be the column to perform the merging on to preserve the integrity of the data.

I'm going to be performing the merging one at a time. Following any merge I'll be using `.head()`,`.info()`, or `.sample()` methods to verify there are not any errors or inaccuracies. 

In [117]:
# Additional dataframes 
games = pd.read_csv("steam.csv")
description = pd.read_csv("steam_description_data.csv")
media_data = pd.read_csv("steam_media_data.csv")
optional = pd.read_csv("steam_optional.csv")
packages = pd.read_csv("steam_packages_info.csv")
requirements = pd.read_csv("steam_requirements_data.csv")
support = pd.read_csv("steam_support_info.csv")
tags = pd.read_csv("steamspy_tag_data.csv")


In [132]:
# List of csv files assigned as variables
df_list = [games, description, media_data, optional, packages, requirements, support, tags]

# List of the variable names as strings
df_name_list = ["games:", "description:", "media_data:", "optional:", "packages:", "requirements:", "support:", "tags:"]

# Iterate through both lists using zip to display first 5 rows of each dataframe
# Each item from the "df_name_list" will printed above each dataframe

# Iterate through lists, zip
for item, name in zip(df_list, df_name_list):
    print("\n") # spaces out dataframes
    print(name) # print the name of each dataframe
    display(item.head()) # utilize ipython import to display data



games:


,appid,type,name,required_age,dlc,fullgame,supported_languages,developers,publishers,packages,platforms,categories,genres,achievements,release_date,supported_audio,coming_soon,price,review_score,total_positive,total_negative,rating,owners,average_forever,median_forever,tags
0,10,game,Counter-Strike,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Valve'],['Valve'],"[574941, 7]","['windows', 'mac', 'linux']","['Multi-player', 'PvP', 'Online PvP', 'Shared/...",['Action'],0.0,2000-11-01,"['English', 'French', 'German', 'Italian', 'Ko...",False,8.19,9.0,117261.0,3686.0,95.566768,10000000-20000000,10499,202,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C..."
1,20,game,Team Fortress Classic,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Valve'],['Valve'],[29],"['windows', 'mac', 'linux']","['Multi-player', 'PvP', 'Online PvP', 'Shared/...",['Action'],0.0,1999-04-01,NaN,False,3.99,8.0,3896.0,705.0,81.939532,2000000-5000000,1637,23,"['Action', 'FPS', 'Multiplayer', 'Classic', 'H..."
2,30,game,Day of Defeat,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Sp...",['Valve'],['Valve'],[30],"['windows', 'mac', 'linux']","['Multi-player', 'Valve Anti-Cheat enabled']",['Action'],0.0,2003-05-01,NaN,False,3.99,8.0,2794.0,398.0,84.223637,5000000-10000000,169,11,"['FPS', 'World War II', 'Multiplayer', 'Shoote..."
3,40,game,Deathmatch Classic,0,NaN,NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Valve'],['Valve'],[31],"['windows', 'mac', 'linux']","['Multi-player', 'PvP', 'Online PvP', 'Shared/...",['Action'],0.0,2001-06-01,NaN,False,3.99,6.0,1214.0,308.0,76.485571,5000000-10000000,2632,6,"['Action', 'FPS', 'Classic', 'Multiplayer', 'S..."
4,50,game,Half-Life: Opposing Force,0,NaN,NaN,"['English', 'French', 'German', 'Korean']",['Gearbox Software'],['Valve'],[32],"['windows', 'mac', 'linux']","['Single-player', 'Multi-player', 'Valve Anti-...",['Action'],0.0,1999-11-01,NaN,False,3.99,9.0,11343.0,519.0,92.916082,5000000-10000000,442,250,"['FPS', 'Action', 'Classic', 'Sci-fi', 'Single..."




description:


,appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...




media_data:


...

,appid,pc_requirements,mac_requirements,linux_requirements,pc_minimum,pc_recommended,mac_minimum,mac_recommended
0,10,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN,"OS X Snow Leopard 10.6.3, 1GB RAM, 4GB Hard D...",NaN
1,20,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN,"OS X Snow Leopard 10.6.3, 1GB RAM, 4GB Hard D...",NaN
2,30,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN,"OS X Snow Leopard 10.6.3, 1GB RAM, 4GB Hard D...",NaN
3,40,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN,"OS X Snow Leopard 10.6.3, 1GB RAM, 4GB Hard D...",NaN
4,50,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN,"OS X Snow Leopard 10.6.3, 1GB RAM, 4GB Hard D...",NaN




support:


,appid,website,support_url,support_email
0,10,NaN,http://steamcommunity.com/app/10,NaN
1,30,http://www.dayofdefeat.com/,NaN,NaN
2,50,NaN,https://help.steampowered.com,NaN
3,70,http://www.half-life.com/,http://steamcommunity.com/app/70,NaN
4,80,NaN,http://steamcommunity.com/app/80,NaN




tags:


,appid,1980s,1990s,2.5d,2d,2d_fighter,2d_platformer,360_video,3d,3d_fighter,3d_platformer,3d_vision,4_player_local,4x,6dof,8_bit_music,atv,abstract,action,action_rpg,action_rts,action_roguelike,action_adventure,addictive,adventure,agriculture,aliens,alternate_history,ambient,america,animation_&_modeling,anime,arcade,archery,arena_shooter,artificial_intelligence,assassin,asymmetric_vr,asynchronous_multiplayer,atmospheric,audio_production,auto_battler,automation,automobile_sim,bmx,base_building,baseball,based_on_a_novel,basketball,batman,battle_royale,beat_em_up,beautiful,benchmark,bikes,blood,board_game,boss_rush,bowling,boxing,building,bullet_hell,bullet_time,crpg,capitalism,card_battler,card_game,cartoon,cartoony,casual,cats,character_action_game,character_customization,chess,choices_matter,choose_your_own_adventure,cinematic,city_builder,class_based,classic,clicker,co_op,co_op_campaign,cold_war,collectathon,colony_sim,colorful,combat,combat_racing,comedy,comic_book,competitive,conspiracy,controller,conversation,cooking,crafting,creature_collector,crime,crowdfunded,cult_classic,cute,cyberpunk,cycling,dark,dark_comedy,dark_fantasy,dark_humor,dating_sim,deckbuilding,demons,design_&_illustration,destruction,detective,difficult,dinosaurs,diplomacy,documentary,dog,dragons,drama,driving,dungeon_crawler,dungeons_&_dragons,dynamic_narration,dystopian_,early_access,economy,education,electronic,electronic_music,emotional,epic,episodic,escape_room,experience,experimental,exploration,fmv,fps,faith,family_friendly,fantasy,farming,farming_sim,fast_paced,feature_film,female_protagonist,fighting,first_person,fishing,flight,football,foreign,free_to_play,funny,futuristic,gambling,game_development,gamemaker,games_workshop,gaming,god_game,golf,gore,gothic,grand_strategy,great_soundtrack,grid_based_movement,gun_customization,hack_and_slash,hacking,hand_drawn,hardware,heist,hentai,hero_shooter,hex_grid,hidden_object,historical,hockey,horror,horses,hunting,idler,illuminati,immersive,immersive_sim,indie,instrumental_music,intentionally_awkward_controls,interactive_fiction,inventory_management,investigation,isometric,jrpg,jet,kickstarter,lego,lgbtq+,lemmings,level_editor,life_sim,linear,local_co_op,local_multiplayer,logic,loot,looter_shooter,lore_rich,lovecraftian,mmorpg,moba,magic,management,mars,martial_arts,massively_multiplayer,masterpiece,match_3,mature,mechs,medical_sim,medieval,memes,metroidvania,military,mini_golf,minigames,minimalist,mining,mod,moddable,modern,motocross,motorbike,mouse_only,movie,multiplayer,multiple_endings,music,music_based_procedural_generation,mystery,mystery_dungeon,mythology,nsfw,narration,narrative,nature,naval,naval_combat,ninja,noir,nonlinear,nostalgia,nudity,offroad,old_school,on_rails_shooter,online_co_op,open_world,open_world_survival_craft,otome,outbreak_sim,parkour,parody_,party,party_game,party_based_rpg,perma_death,philisophical,philosophical,photo_editing,physics,pinball,pirates,pixel_graphics,platformer,point_&_click,political,political_sim,politics,pool,post_apocalyptic,precision_platformer,procedural_generation,programming,psychedelic,psychological,psychological_horror,puzzle,puzzle_platformer,pve,pvp,quick_time_events,rpg,rpgmaker,rts,racing,real_time_tactics,real_time,real_time_with_pause,realistic,reboot,relaxing,remake,replay_value,resource_management,retro,rhythm,robots,rock_music,rogue_like,rogue_lite,roguelike_deckbuilder,roguevania,romance,rome,runner,sailing,sandbox,satire,sci_fi,science,score_attack,sequel,sexual_content,shoot_em_up,shooter,short,side_scroller,silent_protagonist,simulation,singleplayer,skateboarding,skating,skiing,sniper,snow,snowboarding,soccer,social_deduction,software,software_training,sokoban,solitaire,souls_like,soundtrack,space,space_sim,spaceships,spectacle_fighter,spelling,split_screen,sports,star_wars,stealth,steam_machine,steampunk,story_rich,strategy,strategy_rpg,stylized,submarine,superhero,supernatural,surreal,survival,survival_horror,swordplay,tabletop,tactical,tac

### Dataframes to use

1. description: description of each game
2. games: majority of numeric data to include price, release date, rating, etc..
3. optional: some extra descriptions, and includes metacritic score data for each app
4. packages: most useful column is "is_recurring_subscription"

The Kaggle page for these datasets were contained in one zip folder when I downloaded the datasets. As seen above, the author supplied more datasets, but I believe these four datasets will suffice for analysis.

During the process of merging dataframes I noticed that I kept ending up with extra rows. The maximum dataframe length should not have exceeded 52,284 rows. This happened after merging the `packages` dataframe, because this set of data had duplicated game IDs, thus it would create extra rows. To fix this problem I used the pandas method `.drop_duplicates()` with the subset set to `appid` on the `packages` dataframe before merging to the main dataframe `trends_clean`.

In [35]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102504 entries, 0 to 102503
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   appid                102504 non-null  int64  
 1   type                 102504 non-null  object 
 2   name                 102504 non-null  object 
 3   required_age         102504 non-null  int64  
 4   dlc                  9696 non-null    object 
 5   fullgame             34607 non-null   object 
 6   supported_languages  102352 non-null  object 
 7   developers           102463 non-null  object 
 8   publishers           102464 non-null  object 
 9   packages             81153 non-null   object 
 10  platforms            102504 non-null  object 
 11  categories           102398 non-null  object 
 12  genres               102311 non-null  object 
 13  achievements         102504 non-null  float64
 14  release_date         95676 non-null   object 
 15  supported_audio  

In [86]:
# Frequency of type (game, dlc, ads, music) in games dataframe
games["type"].value_counts()

game           67870
dlc            34632
advertising        1
music              1
Name: type, dtype: int64

In [38]:
#trends = pd.read_csv("steam_charts_clean.csv")

In [39]:
# Merging with description dataframe
trends_clean = trends_clean.merge(description, how="left",left_on="app_id", right_on="appid")

In [40]:
trends_clean.head()

,Month,average_players,peak_players,app_id,game,player_gain,player_gain_percentage,appid,detailed_description,about_the_game,short_description
0,2022-06-30,570825.94,923996,730,Counter-Strike: Global Offensive,5864.20,1.04,730.0,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...
1,2022-05-31,564961.70,923996,730,Counter-Strike: Global Offensive,-4021.46,-0.71,730.0,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...
2,2022-04-30,568983.16,1013237,730,Counter-Strike: Global Offensive,-12506.53,-2.15,730.0,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...
3,2022-03-31,581489.69,987993,730,Counter-Strike: Global Offensive,-53148.71,-8.37,730.0,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...
4,2022-02-28,634638.40,995163,730,Counter-Strike: Global Offensive,32262.13,5.36,730.0,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...


In [41]:
trends_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52284 entries, 0 to 52283
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Month                   52284 non-null  datetime64[ns]
 1   average_players         52284 non-null  float64       
 2   peak_players            52284 non-null  int64         
 3   app_id                  52284 non-null  int64         
 4   game                    52284 non-null  object        
 5   player_gain             52284 non-null  float64       
 6   player_gain_percentage  52284 non-null  float64       
 7   appid                   47466 non-null  float64       
 8   detailed_description    47466 non-null  object        
 9   about_the_game          47466 non-null  object        
 10  short_description       47466 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 4.8+ MB


In [42]:
# Merging with games dataframe
trends_clean = trends_clean.merge(games, how="left",left_on="app_id", right_on="appid")

In [43]:
trends_clean.head()

,Month,average_players,peak_players,app_id,game,player_gain,player_gain_percentage,appid_x,detailed_description,about_the_game,short_description,appid_y,type,name,required_age,dlc,fullgame,supported_languages,developers,publishers,packages,platforms,categories,genres,achievements,release_date,supported_audio,coming_soon,price,review_score,total_positive,total_negative,rating,owners,average_forever,median_forever,tags
0,2022-06-30,570825.94,923996,730,Counter-Strike: Global Offensive,5864.20,1.04,730.0,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,730.0,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive..."
1,2022-05-31,564961.70,923996,730,Counter-Strike: Global Offensive,-4021.46,-0.71,730.0,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,730.0,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive..."
2,2022-04-30,568983.16,1013237,730,Counter-Strike: Global Offensive,-12506.53,-2.15,730.0,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,730.0,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive..."
3,2022-03-31,581489.69,987993,730,Counter-Strike: Global Offensive,-53148.71,-8.37,730.0,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,730.0,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive..."
4,2022-02-28,634638.40,995163,730,Counter-Strike: Global Offensive,32262.13,5.36,730.0,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,730.0,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive..."


In [44]:
trends_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52284 entries, 0 to 52283
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Month                   52284 non-null  datetime64[ns]
 1   average_players         52284 non-null  float64       
 2   peak_players            52284 non-null  int64         
 3   app_id                  52284 non-null  int64         
 4   game                    52284 non-null  object        
 5   player_gain             52284 non-null  float64       
 6   player_gain_percentage  52284 non-null  float64       
 7   appid_x                 47466 non-null  float64       
 8   detailed_description    47466 non-null  object        
 9   about_the_game          47466 non-null  object        
 10  short_description       47466 non-null  object        
 11  appid_y                 47466 non-null  float64       
 12  type                    47466 non-null  object

The `.sample()` method returns random rows. You can see that games in original dataset (`trends_clean`) are matching up correctly with the additional datasets that were merged to it. If you search the game on the Steam store it will display the categories, developers and publishers which correctly match here in this new dataframe.

In [45]:
trends_clean.sample(10)

,Month,average_players,peak_players,app_id,game,player_gain,player_gain_percentage,appid_x,detailed_description,about_the_game,short_description,appid_y,type,name,required_age,dlc,fullgame,supported_languages,developers,publishers,packages,platforms,categories,genres,achievements,release_date,supported_audio,coming_soon,price,review_score,total_positive,total_negative,rating,owners,average_forever,median_forever,tags
25881,2022-01-31,1592.97,2826,571740,Golf It!,297.56,22.97,571740.0,<strong>Golf It!</strong> is a multiplayer Min...,<strong>Golf It!</strong> is a multiplayer Min...,Golf It! is a multiplayer Minigolf game with f...,571740.0,game,Golf It!,0.0,NaN,NaN,"['Czech', 'English', 'French', 'German', 'Japa...",['Perfuse Entertainment'],['Perfuse Entertainment'],[145163],"['windows', 'mac']","['Single-player', 'Multi-player', 'PvP', 'Onli...","['Casual', 'Indie', 'Simulation', 'Sports', 'E...",41.0,2017-02-17,NaN,False,8.99,8.0,13383.0,1531.0,87.532658,1000000-2000000,444.0,271.0,"['Mini Golf', 'Multiplayer', 'Golf', 'Casual',..."
44991,2016-01-31,404.86,1325,232890,Stronghold Crusader 2,36.57,9.93,232890.0,"<h1>Digital Deluxe Edition</h1><p><img src=""ht...","<img src=""https://cdn.akamai.steamstatic.com/s...",Stronghold Crusader 2 is the long awaited sequ...,232890.0,game,Stronghold Crusader 2,0.0,"[317590, 317591, 348990, 349000, 349230, 349231]",NaN,"['English', 'French', 'German', 'Italian', 'Ja...",['FireFly Studios'],['FireFly Studios'],"[50936, 50937, 81764]",['windows'],"['Single-player', 'Multi-player', 'PvP', 'Onli...","['Simulation', 'Strategy']",81.0,2014-09-22,"['English', 'French', 'German', 'Italian', 'Po...",False,29.99,6.0,6239.0,2516.0,69.879033,1000000-2000000,2825.0,311.0,"['Strategy', 'Medieval', 'City Builder', 'RTS'..."
50658,2014-11-30,615.36,1145,234330,Marvel Puzzle Quest,-42.23,-6.42,234330.0,Marvel Comics Super Heroes come to life in thi...,Marvel Comics Super Heroes come to life in thi...,The best in Match 3 puzzle gameplay meets Marv...,234330.0,game,MARVEL Puzzle Quest,0.0,"[265130, 265131, 265132, 1450690]",NaN,"['English', 'French', 'German', 'Italian', 'Ja...",['Demiurge Studios'],['D3 Go!'],NaN,['windows'],"['Single-player', 'Multi-player', 'PvP', 'Onli...","['Casual', 'Free to Play', 'RPG']",0.0,2013-12-05,"['English', 'French', 'German', 'Italian', 'Ja...",False,0.00,5.0,14.0,8.0,58.330237,500000-1000000,105.0,79.0,"['Puzzle', 'Match 3', 'RPG', 'Superhero', 'Fre..."
7492,2021-11-30,7109.91,12818,444090,Paladins,-466.20,-6.15,444090.0,<h1>FREE Team Fortress 2 Barik Skin</h1><p><st...,"<img src=""https://cdn.akamai.steamstatic.com/s...","Join 50+ million players in Paladins, the free...",444090.0,game,Paladins®,0.0,"[1517480, 1517481, 1721880, 1843930, 1843931, ...",NaN,"['English', 'French', 'German', 'Japanese', 'P...",['Evil Mojo Games'],['Hi-Rez Studios'],[216127],['windows'],"['Multi-player', 'PvP', 'Online PvP', 'Steam A...","['Action', 'Free to Play']",58.0,2018-05-08,['English'],False,0.00,8.0,7668.0,1559.0,80.984288,5000000-10000000,3525.0,492.0,"['Hero Shooter', 'Free to Play', 'Multiplayer'..."
18312,2020-07-31,3772.18,6436,629760,MORDHAU,21.62,0.58,629760.0,<strong>MORDHAU</strong> is a medieval first &...,<strong>MORDHAU</strong> is a medieval first &...,MORDHAU is a multiplayer medieval slasher. Cre...,629760.0,game,MORDHAU,0.0,"[1049020, 1836060, 1836061]",NaN,"['English', 'French', 'German', 'Italian', 'Ko...",['Triternion'],['Triternion'],"[171237, 362163]",['windows'],"['Multi-player', 'PvP', 'Online PvP', 'Co-op',...","['Action', 'Indie']",38.0,2019-04-29,['English'],False,24.99,8.0,67519.0,15054.0,80.717177,2000000-5000000,3594.0,796.0,"['Medieval', 'Multiplayer', 'First-Person', 'S..."
35368,2020-08-31,403.97,767,753640,Outer Wilds,-605.20,-59.97,753640.0,Winner of Best Game at the 2020 BAFTA Games Aw...,Winner of Best Game at the 2020 BAFTA Games Aw...,"Named Game of the Year 2019 by Giant Bomb, Pol...",753640.0,game,Outer Wilds,0.0,"[1286760, 1622100]",NaN,"['Engl

In [48]:
# Merging with "optional" dataframe
trends_clean = trends_clean.merge(optional, how="left",left_on="app_id", right_on="appid")

In [49]:
# Displays dataframe columns only
trends_clean.columns

Index(['Month', 'average_players', 'peak_players', 'app_id', 'game',
       'player_gain', 'player_gain_percentage', 'appid_x',
       'detailed_description', 'about_the_game', 'short_description',
       'appid_y', 'type', 'name', 'required_age', 'dlc', 'fullgame',
       'supported_languages', 'developers', 'publishers', 'packages',
       'platforms', 'categories', 'genres', 'achievements', 'release_date',
       'supported_audio', 'coming_soon', 'price', 'review_score',
       'total_positive', 'total_negative', 'rating', 'owners',
       'average_forever', 'median_forever', 'tags', 'appid', 'drm_notice',
       'ext_user_account_notice', 'demos', 'content_descriptors',
       'metacritic_score', 'metacritic_url'],
      dtype='object')

I'll drop some columns that I know we won't need. Some of them are duplicated "appid" columns that are produced after merging data. Further below there will be more data cleaning.

In [50]:
# List of some columns to remove
columns_to_drop = ['appid_x','appid_y','appid','drm_notice','ext_user_account_notice']

# Drop columns
trends_clean = trends_clean.drop(columns=columns_to_drop)

In [51]:
trends_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52284 entries, 0 to 52283
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Month                   52284 non-null  datetime64[ns]
 1   average_players         52284 non-null  float64       
 2   peak_players            52284 non-null  int64         
 3   app_id                  52284 non-null  int64         
 4   game                    52284 non-null  object        
 5   player_gain             52284 non-null  float64       
 6   player_gain_percentage  52284 non-null  float64       
 7   detailed_description    47466 non-null  object        
 8   about_the_game          47466 non-null  object        
 9   short_description       47466 non-null  object        
 10  type                    47466 non-null  object        
 11  name                    47466 non-null  object        
 12  required_age            47466 non-null  float6

In [61]:
# This is a preview of how the "packages" data is adding extra rows
trends_clean.merge(packages, how="left",left_on="app_id", right_on="appid").tail()

,Month,average_players,peak_players,app_id,game,player_gain,player_gain_percentage,detailed_description,about_the_game,short_description,type_x,name,required_age,dlc,fullgame,supported_languages,developers,publishers,packages,platforms,categories,genres,achievements,release_date,supported_audio,coming_soon,price,review_score,total_positive,total_negative,rating,owners,average_forever,median_forever,tags,demos,content_descriptors,metacritic_score,metacritic_url,appid,type_y,title,is_recurring_subscription,subs
0,2022-06-30,570825.94,923996,730,Counter-Strike: Global Offensive,5864.20,1.04,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.00,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",NaN,Includes intense violence and blood.,83.0,https://www.metacritic.com/game/pc/counter-str...,730.0,default,Buy Counter-Strike: Global Offensive,False,"[{'packageid': 298963, 'percent_savings_text':..."
1,2022-05-31,564961.70,923996,730,Counter-Strike: Global Offensive,-4021.46,-0.71,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.00,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",NaN,Includes intense violence and blood.,83.0,https://www.metacritic.com/game/pc/counter-str...,730.0,default,Buy Counter-Strike: Global Offensive,False,"[{'packageid': 298963, 'percent_savings_text':..."
2,2022-04-30,568983.16,1013237,730,Counter-Strike: Global Offensive,-12506.53,-2.15,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.00,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",NaN,Includes intense violence and blood.,83.0,https://www.metacritic.com/game/pc/counter-str...,730.0,default,Buy Counter-Strike: Global Offensive,False,"[{'packageid': 298963, 'percent_savings_text':..."
3,2022-03-31,581489.69,987993,730,Counter-Strike: Global Offensive,-53148.71,-8.37,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.00,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",NaN,Includes intense violence and blood.,83.0,https://www.metacritic.com/game/pc/counter-str...,730.0,default,Buy Cou

In [63]:
# Drop duplicates from the "appid" column in the packages dataframe
packages = packages.drop_duplicates(subset=["appid"])

In [65]:
# Merging with games dataframe
trends_clean = trends_clean.merge(packages, how="left",left_on="app_id", right_on="appid")

In [67]:
# Final dataframe info
# Memory usage over 500mb
trends_clean.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52284 entries, 0 to 52283
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Month                      52284 non-null  datetime64[ns]
 1   average_players            52284 non-null  float64       
 2   peak_players               52284 non-null  int64         
 3   app_id                     52284 non-null  int64         
 4   game                       52284 non-null  object        
 5   player_gain                52284 non-null  float64       
 6   player_gain_percentage     52284 non-null  float64       
 7   detailed_description       47466 non-null  object        
 8   about_the_game             47466 non-null  object        
 9   short_description          47466 non-null  object        
 10  type_x                     47466 non-null  object        
 11  name                       47466 non-null  object        
 12  requ

## Further data cleaning to be performed

The player trends (`steam_charts.csv`) data was cleaned earlier. I wanted to get that accomplished before merging more data, because it's easier to work with only a few columns in one dataset rather than trying to wait till merging multiple datasets with dozens of columns.

Sometimes when merging data you end up with missing data or unneccesary columns. Before creating a new csv file from this new data some of these nulls, and extra columns need to be dealt with.

Below you can use the following code to show all missing data for each column in descending order. Typically, I scrutinize columns with over 50% missing data. The first three highest columns will be dropped because they contain over 90% missing values.



In [70]:
# Percent of missing data, sorted descending order
(trends_clean.apply(pd.isnull).sum()/trends_clean.shape[0] * 100).sort_values(ascending=False)

fullgame                     100.000000
content_descriptors           92.976819
demos                         91.500268
metacritic_score              50.418866
metacritic_url                50.418866
dlc                           42.869712
supported_audio               41.840716
subs                          23.521536
appid                         23.521536
title                         23.521536
is_recurring_subscription     23.521536
type_y                        23.521536
packages                      22.850203
price                         10.205799
tags                           9.968633
release_date                   9.874914
genres                         9.656874
total_positive                 9.215056
review_score                   9.215056
rating                         9.215056
total_negative                 9.215056
owners                         9.215056
coming_soon                    9.215056
median_forever                 9.215056
average_forever                9.215056


In [72]:
trends_clean.columns

Index(['Month', 'average_players', 'peak_players', 'app_id', 'game',
       'player_gain', 'player_gain_percentage', 'detailed_description',
       'about_the_game', 'short_description', 'type_x', 'name', 'required_age',
       'dlc', 'fullgame', 'supported_languages', 'developers', 'publishers',
       'packages', 'platforms', 'categories', 'genres', 'achievements',
       'release_date', 'supported_audio', 'coming_soon', 'price',
       'review_score', 'total_positive', 'total_negative', 'rating', 'owners',
       'average_forever', 'median_forever', 'tags', 'demos',
       'content_descriptors', 'metacritic_score', 'metacritic_url', 'appid',
       'type_y', 'title', 'is_recurring_subscription', 'subs'],
      dtype='object')

In [81]:
# Show the difference between the different descriptions
trends_clean[['detailed_description','about_the_game', 'short_description']].sample(10)

,detailed_description,about_the_game,short_description
43003,"<img src=""https://cdn.akamai.steamstatic.com/s...","<img src=""https://cdn.akamai.steamstatic.com/s...",100% Orange Juice is a digital multiplayer boa...
19425,Sid Meier's Civilization IV®: Beyond the Sword...,Sid Meier's Civilization IV®: Beyond the Sword...,Sid Meier's Civilization IV®: Beyond the Sword...
37208,"<h2 class=""bb_tag"">The Classic Adventure Conti...","<h2 class=""bb_tag"">The Classic Adventure Conti...",Rediscover the beloved RPG classic— now enhanc...
45214,NaN,NaN,NaN
3911,The Flagship Turn-Based Strategy Game Returns<...,The Flagship Turn-Based Strategy Game Returns<...,"Create, discover, and download new player-crea..."
50653,Marvel Comics Super Heroes come to life in thi...,Marvel Comics Super Heroes come to life in thi...,The best in Match 3 puzzle gameplay meets Marv...
39934,Play the top-rated tower defense franchise in ...,Play the top-rated tower defense franchise in ...,Go head to head with other players in a Bloon-...
27211,<h1>GOLD EDITION</h1><p>The Gold Edition inclu...,"London, 1868. In the heart of the Industrial R...","London, 1868. In the heart of the Industrial R..."
45101,Gang Beasts is a silly multiplayer party game ...,Gang Beasts is a silly multiplayer party game ...,Gang Beasts is a silly multiplayer party game ...
41542,"<img src=""https://cdn.akamai.steamstatic.com/s...","<img src=""https://cdn.akamai.steamstatic.com/s...","A massively multiplayer grand strategy game, S..."


Setting the random state to "12" for `.sample()` shows how merging on the `app_id` was an appropriate choice. The game names are the same, however you can see how they may be spelled slightly differently.

For example, "Sid Meier's Civilization IV: Beyond the Sword" is the same game as "Civilization IV: Beyond the Sword". They have the same app ID but different datasets add or remove words or use different punctuation. Not merging on the ID would have produced too many inconsistencies.

I spent several minutes reviewing the columns to determine if any more needed to be removed. While having extra data is great, more data does not automatically mean it's useful. Without going into too much detail for each variable or column, the ones that will be dropped below are not going to be relevant to the analysis. They're mostly redundent columns or may contain a single value that does not add anything new.

In [105]:
# Game names match but there are differences
trends_clean[["game","name"]].sample(10,random_state=12)

,game,name
5390,Soundpad,NaN
19346,Sid Meier's Civilization IV: Beyond the Sword,Civilization IV: Beyond the Sword
13068,A Dance of Fire and Ice,A Dance of Fire and Ice
38022,The Elder Scrolls III: Morrowind,The Elder Scrolls III: Morrowind® Game of the ...
21967,FTL: Faster Than Light,FTL: Faster Than Light
13158,Crossout,Crossout
14856,Grand Theft Auto IV: The Complete Edition,Grand Theft Auto IV: Complete Edition
35426,UBOAT,UBOAT
35388,She Will Punish Them,She Will Punish Them
50215,Bejeweled 3,Bejeweled® 3


In [82]:
trends_clean.head()

,Month,average_players,peak_players,app_id,game,player_gain,player_gain_percentage,detailed_description,about_the_game,short_description,type_x,name,required_age,dlc,fullgame,supported_languages,developers,publishers,packages,platforms,categories,genres,achievements,release_date,supported_audio,coming_soon,price,review_score,total_positive,total_negative,rating,owners,average_forever,median_forever,tags,demos,content_descriptors,metacritic_score,metacritic_url,appid,type_y,title,is_recurring_subscription,subs
0,2022-06-30,570825.94,923996,730,Counter-Strike: Global Offensive,5864.20,1.04,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",NaN,Includes intense violence and blood.,83.0,https://www.metacritic.com/game/pc/counter-str...,730.0,default,Buy Counter-Strike: Global Offensive,False,"[{'packageid': 298963, 'percent_savings_text':..."
1,2022-05-31,564961.70,923996,730,Counter-Strike: Global Offensive,-4021.46,-0.71,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",NaN,Includes intense violence and blood.,83.0,https://www.metacritic.com/game/pc/counter-str...,730.0,default,Buy Counter-Strike: Global Offensive,False,"[{'packageid': 298963, 'percent_savings_text':..."
2,2022-04-30,568983.16,1013237,730,Counter-Strike: Global Offensive,-12506.53,-2.15,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",NaN,Includes intense violence and blood.,83.0,https://www.metacritic.com/game/pc/counter-str...,730.0,default,Buy Counter-Strike: Global Offensive,False,"[{'packageid': 298963, 'percent_savings_text':..."
3,2022-03-31,581489.69,987993,730,Counter-Strike: Global Offensive,-53148.71,-8.37,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,Counter-Strike: Global Offensive (CS: GO) expa...,game,Counter-Strike: Global Offensive,0.0,[1766730],NaN,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"[329385, 298963, 54029]","['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",NaN,Includes intense violence and blood.,83.0,https://www.metacritic.com/game/pc/counter-str...,730.0,default,Buy Counter

In [107]:
# More columns to drop
list_of_cols = [
    "fullgame",
    "content_descriptors",
    "demos",
    "appid",
    "type_y",
    "type_x",
    "detailed_description",
    "about_the_game",
    "packages",
    "fullgame",
    "subs",
    "title",
    "name"
    ]

trends_clean = trends_clean.drop(list_of_cols, axis = 1)

In [108]:
trends_clean.head()

,Month,average_players,peak_players,app_id,game,player_gain,player_gain_percentage,short_description,required_age,dlc,supported_languages,developers,publishers,platforms,categories,genres,achievements,release_date,supported_audio,coming_soon,price,review_score,total_positive,total_negative,rating,owners,average_forever,median_forever,tags,metacritic_score,metacritic_url,is_recurring_subscription
0,2022-06-30,570825.94,923996,730,Counter-Strike: Global Offensive,5864.20,1.04,Counter-Strike: Global Offensive (CS: GO) expa...,0.0,[1766730],"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",83.0,https://www.metacritic.com/game/pc/counter-str...,False
1,2022-05-31,564961.70,923996,730,Counter-Strike: Global Offensive,-4021.46,-0.71,Counter-Strike: Global Offensive (CS: GO) expa...,0.0,[1766730],"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",83.0,https://www.metacritic.com/game/pc/counter-str...,False
2,2022-04-30,568983.16,1013237,730,Counter-Strike: Global Offensive,-12506.53,-2.15,Counter-Strike: Global Offensive (CS: GO) expa...,0.0,[1766730],"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",83.0,https://www.metacritic.com/game/pc/counter-str...,False
3,2022-03-31,581489.69,987993,730,Counter-Strike: Global Offensive,-53148.71,-8.37,Counter-Strike: Global Offensive (CS: GO) expa...,0.0,[1766730],"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",83.0,https://www.metacritic.com/game/pc/counter-str...,False
4,2022-02-28,634638.40,995163,730,Counter-Strike: Global Offensive,32262.13,5.36,Counter-Strike: Global Offensive (CS: GO) expa...,0.0,[1766730],"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.0,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",83.0,https://www.metacritic.com/game/pc/counter-str...,False


The author of this data states in their description on Kaggle that the price of the games are in Euros. Valve is an American company, but the author is probably from Europe, thus when they collected the data it was in their local currency.

The author last updated the collection of datasets on May 25, 2022, therefore I will be using the conversion rate of that day.

**Wednesday 25, May 2022 conversion:**

**€1 EUR = $1.0689 US dollar**

I'll finalize the *majority* of the data cleaning by converting the price to US dollars, and renaming a few columns.

In [110]:
#Wednesday 25 May 2022	€1 EUR = $1.0689
trends_clean["price"] = trends_clean["price"] * 1.0689

In [113]:
# Dictionary of columns to rename
col_rename = {
    "price":"price_usd",
    "Month":"month",
    }

# Rename columns
combined_df_trends = trends_clean.rename(columns=col_rename).copy() # <-- Set copy

In [114]:
# Combined dataframes to a single csv file
#combined_df_trends.to_csv("steam_charts_combined.csv",index=False)

In [115]:
pd.read_csv("steam_charts_combined.csv")

,month,average_players,peak_players,app_id,game,player_gain,player_gain_percentage,short_description,required_age,dlc,supported_languages,developers,publishers,platforms,categories,genres,achievements,release_date,supported_audio,coming_soon,price_usd,review_score,total_positive,total_negative,rating,owners,average_forever,median_forever,tags,metacritic_score,metacritic_url,is_recurring_subscription
0,2022-06-30,570825.94,923996,730,Counter-Strike: Global Offensive,5864.20,1.04,Counter-Strike: Global Offensive (CS: GO) expa...,0.0,[1766730],"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.000000,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",83.0,https://www.metacritic.com/game/pc/counter-str...,False
1,2022-05-31,564961.70,923996,730,Counter-Strike: Global Offensive,-4021.46,-0.71,Counter-Strike: Global Offensive (CS: GO) expa...,0.0,[1766730],"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.000000,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",83.0,https://www.metacritic.com/game/pc/counter-str...,False
2,2022-04-30,568983.16,1013237,730,Counter-Strike: Global Offensive,-12506.53,-2.15,Counter-Strike: Global Offensive (CS: GO) expa...,0.0,[1766730],"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.000000,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",83.0,https://www.metacritic.com/game/pc/counter-str...,False
3,2022-03-31,581489.69,987993,730,Counter-Strike: Global Offensive,-53148.71,-8.37,Counter-Strike: Global Offensive (CS: GO) expa...,0.0,[1766730],"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.000000,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",83.0,https://www.metacritic.com/game/pc/counter-str...,False
4,2022-02-28,634638.40,995163,730,Counter-Strike: Global Offensive,32262.13,5.36,Counter-Strike: Global Offensive (CS: GO) expa...,0.0,[1766730],"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...","['Valve', 'Hidden Path Entertainment']",['Valve'],"['windows', 'mac', 'linux']","['Multi-player', 'Steam Achievements', 'Full c...","['Action', 'Free to Play']",167.0,2012-08-21,[' English'],False,0.000000,8.0,2949363.0,439344.0,86.634302,50000000-100000000,30053.0,6104.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",83.0,https://www.metacritic.com/game/pc/counter-str...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52279,2020-06-30,444.60,1019,625960,Stoneshard,-37.79,-7.83,Stoneshard is a challenging turn-based RPG set...,0.0,[1230230],"['English', 'French', 'German', 'Italian', 'Ja...",['Ink Stains Games'],['HypeTrain Digital'],"['windows', 'linux']","['Single-player', 'Captions available', 'Remot...","['Adventure', 'Indie', 'RPG', 'Strategy', 'Ear...",54.0,2020-02-06,NaN,False,26.711811,6.0,15281.0,4705.0,75.116030,500000-1000000,1338.0,769.0,"['RPG', 'Strategy', 'Rogue-like', 'Open World ...",NaN,NaN,False
52280,2020-05-31,482.40,1400,625960